In [1]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def read_beta_daily(db, name, start_date=None, end_date=None, skey=None, interval=None, col=None, return_sdi=True): 
    collection = db[name] 
    # Build projection 
    prj = {'_id': 0} 
    if col is not None: 
        if return_sdi: 
            col = ['skey', 'date'] + col 
        for col_name in col: 
            prj[col_name] = 1 
 
    # Build query 
    query = {} 
    if skey is not None: 
        query['skey'] = {'$in': skey} 
    if interval is not None: 
        query['interval'] = {'$in': interval} 
    if start_date is not None: 
        if end_date is not None: 
            query['date'] = {'$gte': start_date, '$lte': end_date} 
        else: 
            query['date'] = {'$gte': start_date} 
    elif end_date is not None: 
        query['date'] = {'$lte': end_date} 
 
    # Load data 
    cur = collection.find(query, prj) 
    df = pd.DataFrame.from_records(cur) 
    if df.empty: 
        df = pd.DataFrame() 
    else: 
        df = df.sort_values(by=['date','skey']) 
    return df  

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)


import os
import glob
import datetime
import numpy as np
import pandas as pd

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201106'
endDate = '20201216'

readPath = '/mnt/equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

index = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 1000905, 1000852, 1000985])
index = index[['index_id', 'skey', 'date']].reset_index(drop=True)
index = index.rename(columns={'index_id':'indexCat', 'skey':'secid'})
index.loc[index['indexCat'] == 1000985, 'indexCat'] = 1000852
index['ID'] = index['secid']

# addData = pd.DataFrame({'indexCat': [1000300, 1000905, 1000852], 'ID': [1000300, 1000905, 1000852]})
# indexCatData = pd.concat([indexCatData, addData], sort=False).reset_index(drop=True)

# d1 = pd.read_pickle('/mnt/ShareWithServer/stockBeta_L_IC_60d.pkl').reset_index()
# d1['indexCat'] = 1000905
# d1 = d1[(d1['index'] >= int(startDate)) & (d1['index'] <= int(endDate))]
# d2 = pd.read_pickle('/mnt/ShareWithServer/stockBeta_L_IF_60d.pkl').reset_index()
# d2['indexCat'] = 1000300
# d2 = d2[(d2['index'] >= int(startDate)) & (d2['index'] <= int(endDate))]
# d3 = pd.read_pickle('/mnt/ShareWithServer/stockBeta_L_CSI_60d.pkl').reset_index()
# d3['indexCat'] = 1000852
# d3 = d3[(d3['index'] >= int(startDate)) & (d3['index'] <= int(endDate))]
# d = pd.concat([d1, d2, d3]).sort_values(by='index')
# betaData = []
# for i in d.columns[1:-1]:
#     col = [i] + ['index', 'indexCat']
#     re = d[col]
#     re = re.rename(columns={i: 'beta_60', 'index':'date'})
#     re['secid'] = int(i[2:]) + 2000000 if i[:2] == 'SZ' else int(i[2:]) + 1000000
#     betaData += [re]
# betaData = pd.concat(betaData).reset_index(drop=True)
# betaData['date'] = betaData['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').date())

d = read_beta_daily(db1, 'mktbeta', int(startDate), int(endDate))
d1 = d[['skey', 'beta_60d_IF', 'date']]
d1 = d1.rename(columns={'beta_60d_IF':"beta_60"})
d1['indexCat'] = 1000300
d2 = d[['skey', 'beta_60d_IC', 'date']]
d2 = d2.rename(columns={'beta_60d_IC':"beta_60"})
d2['indexCat'] = 1000905
d3 = d[['skey', 'beta_60d_CSI1000', 'date']]
d3 = d3.rename(columns={'beta_60d_CSI1000':"beta_60"})
d3['indexCat'] = 1000852
betaData = pd.concat([d1, d2, d3]).reset_index(drop=True)
betaData['date'] = betaData['date'].apply(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d').date())
betaData = betaData.rename(columns={'skey':'secid'})

for date in np.sort(dateLs):
    
    dateDate = datetime.datetime.strptime(date, '%Y%m%d').date()
    dateBetaData = betaData[betaData['date'] == dateDate]
    
    readPath = '/mnt/equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%date))
    orderLog = orderLog[~orderLog['vai'].isnull()]
    orderLog = orderLog.rename(columns={'mdClockAtArrival': 'caamd'})
    display('There are accounts with duplicated ticks:')
    display(orderLog[orderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep=False)]\
    .groupby(['date', 'colo', 'accCode'])['ars'].size())
    orderLog = orderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                        'orderDirection', 'absOrderSize'], keep='first')
    
    display('There are ticks with orderDirection 0')
    display(orderLog[orderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
                'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

    assert(orderLog[orderLog['updateType'] == 0][orderLog[orderLog['updateType'] == 0]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    try:
        assert(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
    except:
        print('There are orders with all things same except sdd')
        print(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId'], keep=False)])
        assert(orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)][orderLog[(orderLog['updateType'] == 0) & (orderLog['accCode'] != 8856)]\
                                                           .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                        'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
    try:
        assert(sum(orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
    except:
        print('There are orders with same internalId but different orderId other than accCode 8856 case')
        print(orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[orderLog[(orderLog['updateType'] != 0) & (orderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                    'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

    r2 = orderLog[(orderLog['accCode'] != 8856) & (orderLog['orderDirection'] != 0)]
    r1 = orderLog[(orderLog['accCode'] == 8856) & (orderLog['orderDirection'] != 0)]
    r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize']).grouper.group_info[0]
    r1 = r1.sort_values(by=['test', 'clockAtArrival'])
    r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
    r1['vai'] = r1.groupby('test')['vai'].ffill()
    r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    r2 = r2.sort_values(by=['test', 'clockAtArrival'])
    r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
    r2['vai'] = r2.groupby('test')['vai'].ffill()
    assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    try:
        assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
    except:
        a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
        print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
    orderLog = pd.concat([r1, r2])
    del r1
    del r2  
    
    orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
    orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')
    orderLog['caamd'] = orderLog.groupby('order')['caamd'].transform('first')
    orderLog = orderLog[orderLog['firstUpdateType'] == 0]
    
    orderLog['insertNum'] = np.where(orderLog['updateType'] == 0, 1, 0)
    orderLog['insertNum'] = orderLog.groupby(['order'])['insertNum'].transform('sum')
    orderLog = orderLog[orderLog['insertNum'] == 1]
       
    orderLog['innerSeq'] = orderLog.index.values
    targetStockLs = orderLog['secid'].unique()
    orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')
    orderLog['firstClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
        
    assert(orderLog[orderLog['firstUpdateType'] != 0].shape[0] == 0)
    indexCatData = index[index['date'] == int(date)]
    orderLog = pd.merge(orderLog, indexCatData[['secid', 'indexCat']], how='left', on=['secid'], validate='many_to_one')
    orderLog = pd.merge(orderLog, dateBetaData[['secid', 'indexCat', 'beta_60']], how='left', on=['secid', 'indexCat'], validate='many_to_one')
    
    readPath = '/mnt/Kevin_zhenyu/rawData/logs_%s_***'%date
    mdDataSHPath = glob.glob(os.path.join(readPath, 'mdLog_SH***.csv'))[-1]
    mdDataSH = pd.read_csv(mdDataSHPath)
    mdDataSH['ID'] = mdDataSH['StockID'] + 1000000
    mdDataSH['time'] = mdDataSH.time.str.slice(0, 2) + mdDataSH.time.str.slice(3, 5) + mdDataSH.time.str.slice(6, 8) + '000'
    mdDataSH['time'] = mdDataSH['time'].astype('int64')
    mdDataSH['time'] = mdDataSH.groupby(['ID'])['time'].cummax()
    mdDataSH['max_cum_volume'] = mdDataSH.groupby(['StockID'])['cum_volume'].cummax()
    indexData = mdDataSH[mdDataSH['StockID'].isin([300, 852, 905])][['ID', 'sequenceNo', 'close']].reset_index(drop=True)
    mdDataSH = mdDataSH[mdDataSH['StockID'] >= 600000]
    mdDataSH = mdDataSH[(mdDataSH['cum_volume'] > 0) & (mdDataSH['time'] >= 93000000) &\
                        (mdDataSH['cum_volume'] == mdDataSH['max_cum_volume'])]
    mdDataSH = mdDataSH[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]
    
    mdDataSZPath = glob.glob(os.path.join(readPath, 'mdLog_SZ***.csv'))[-1]
    mdDataSZ = pd.read_csv(mdDataSZPath)
    mdDataSZ['ID'] = mdDataSZ['StockID'] + 2000000
    mdDataSZ['time'] = mdDataSZ.time.str.slice(0, 2) + mdDataSZ.time.str.slice(3, 5) + mdDataSZ.time.str.slice(6, 8) + '000'
    mdDataSZ['time'] = mdDataSZ['time'].astype('int64')
    mdDataSZ['time'] = mdDataSZ.groupby(['ID'])['time'].cummax()
    mdDataSZ['max_cum_volume'] = mdDataSZ.groupby(['StockID'])['cum_volume'].cummax()
    mdDataSZ = mdDataSZ[(mdDataSZ['cum_volume'] > 0) & (mdDataSZ['time'] >= 93000000) &\
                        (mdDataSZ['cum_volume'] == mdDataSZ['max_cum_volume'])]
    mdDataSZ = mdDataSZ[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]
    
    mdData = pd.concat([mdDataSH, mdDataSZ, indexData]).reset_index(drop=True)
    mdData = mdData.sort_values(by=['sequenceNo']).reset_index(drop=True)

    addData = pd.DataFrame({'indexCat': [1000300, 1000905, 1000852], 'ID': [1000300, 1000905, 1000852], 
                            'secid':[1000300, 1000905, 1000852]})
    indexCatData = pd.concat([indexCatData, addData], sort=False).reset_index(drop=True)
    mdData = pd.merge(mdData, indexCatData, how='left', on=['ID'], validate='many_to_one')
    mdData = mdData[~mdData['indexCat'].isnull()].reset_index(drop=True)
    mdData['indexClose'] = np.where(mdData['ID'].isin([1000300, 1000852, 1000905]), mdData['close'], np.nan)
    mdData['indexClose'] = mdData.groupby(['indexCat'])['indexClose'].ffill()
    mdData = mdData[~mdData['ID'].isin([1000300, 1000852, 1000905])].reset_index(drop=True)
    
    mdData = mdData.sort_values(by=['ID', 'sequenceNo']).reset_index(drop=True)
    mdData['safeBid1p'] = np.where(mdData['bid1p'] == 0, mdData['ask1p'], mdData['bid1p'])
    mdData['safeAsk1p'] = np.where(mdData['ask1p'] == 0, mdData['bid1p'], mdData['ask1p'])
    mdData['adjMid'] = (mdData['safeBid1p']*mdData['ask1q'] + mdData['safeAsk1p']*mdData['bid1q'])/(mdData['bid1q'] + mdData['ask1q'])
    
    mdData['session'] = np.where(mdData['time'] >= 130000000, 1, 0)
    def findTmValue(clockLs, tm, method='L', buffer=0):
        maxIx = len(clockLs)
        orignIx = np.arange(maxIx)
        if method == 'F':
            ix = np.searchsorted(clockLs, clockLs+(tm-buffer))
            ## if target future index is next tick, mask
            mask = (orignIx == (ix - 1))|(orignIx == ix)|(ix == maxIx)
        elif method == 'L':
            ## if target future index is last tick, mask
            ix = np.searchsorted(clockLs, clockLs-(tm-buffer))
            ix = ix - 1
            ix[ix<0] = 0
            ## !!!ATTENTION: model3 change
            mask = (orignIx == ix) | ((clockLs-(tm-buffer)).values < clockLs.values[0])
        ix[mask] = -1
        return ix

    mdData = mdData.reset_index(drop=True)
    groupAllData = mdData.groupby(['ID', 'session'])
    mdData['sessionStartCLA'] = groupAllData['clockAtArrival'].transform('min')
    mdData['relativeClock'] = mdData['clockAtArrival'] - mdData['sessionStartCLA']
    mdData['trainFlag'] = np.where(mdData['relativeClock'] > 179.5*1e6, 1, 0)
    mdData['index'] = mdData.index.values
    mdData['sessionStartIx'] = groupAllData['index'].transform('min')
    for tm in [30, 90, 300]:
        tmCol = 'F{}s_ix'.format(tm)
        mdData[tmCol] = groupAllData['relativeClock'].transform(lambda x: findTmValue(x, tm*1e6, 'F', 5*1e5)).astype(int)
    nearLimit = ((mdData.ask5q.values == 0) | (mdData.bid5q.values == 0))
    
    for tm in [30, 90, 300]:
        tmIx = mdData['F{}s_ix'.format(tm)].values + mdData['sessionStartIx'].values
        adjMid_tm = mdData['adjMid'].values[tmIx]
        adjMid_tm[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['adjMid_F{}s'.format(tm)] = adjMid_tm

    for tm in [30, 90, 300]:
        tmIx = mdData['F{}s_ix'.format(tm)].values + mdData['sessionStartIx'].values
        adjMid_tm = mdData['indexClose'].values[tmIx]
        adjMid_tm[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['indexClose_F{}s'.format(tm)] = adjMid_tm
    
    mdData = mdData[mdData['ID'].isin(targetStockLs)]
    mdStartPos = mdData.drop_duplicates(subset=['ID', 'cum_volume'], keep='last')
    mdStartPos = mdStartPos[['ID', 'cum_volume', 'clockAtArrival']].reset_index(drop=True)
    mdStartPos.columns = ['secid', 'vai', 'mdStartClock']
    mdStartPos['isOrder'] = 0
    tradeStartPos = orderLog[orderLog['updateType'] == 0][['secid', 'vai', 'order']].reset_index(drop=True)
    tradeStartPos['isOrder'] = 1
    tradeStartPos = pd.concat([mdStartPos, tradeStartPos], sort=False)
    tradeStartPos = tradeStartPos.sort_values(by=['secid', 'vai', 'isOrder'])
    tradeStartPos['mdStartClock'] = tradeStartPos.groupby(['secid'])['mdStartClock'].ffill()
    tradeStartPos['mdStartClock'] = tradeStartPos.groupby(['secid'])['mdStartClock'].backfill()
    tradeStartPos = tradeStartPos[tradeStartPos['isOrder'] == 1][['secid', 'vai', 'order', 'mdStartClock']]
    
    orderLog = pd.merge(orderLog, tradeStartPos[['order', 'mdStartClock']], how='left', on=['order'], validate='many_to_one')
    orderLog['mdClockAtArrival'] = orderLog['clockAtArrival'] - orderLog['caamd'] + orderLog['mdStartClock']
    
    tradeData = orderLog[['secid', 'mdClockAtArrival', 'innerSeq']].reset_index(drop=True)
    tradeData.columns = ['ID', 'clockAtArrival', 'innerSeq']
    tradeData['isOrder'] = 1
    
    mdData = pd.concat([mdData, tradeData], sort=False)
    mdData = mdData.sort_values(by=['ID', 'clockAtArrival', 'isOrder', 'innerSeq']).reset_index(drop=True)
    for col in ['indexClose', 'adjMid_F30s', 'indexClose_F30s', 'adjMid_F90s', 'indexClose_F90s',
                'adjMid_F300s', 'indexClose_F300s']:
        mdData[col] = mdData.groupby(['ID'])[col].backfill()
        mdData[col] = mdData.groupby(['ID'])[col].ffill()

    tradeData = mdData[mdData['isOrder'] == 1][['ID', 'innerSeq', 'adjMid_F30s', 'adjMid_F90s', 'adjMid_F300s',
                                                'indexClose', 'indexClose_F30s', 'indexClose_F90s', 'indexClose_F300s']].reset_index(drop=True)
    tradeData = tradeData.rename(columns={'ID': 'secid'})
    orderLog = pd.merge(orderLog, tradeData, how='left', on=['secid', 'innerSeq'], validate='one_to_one')

    savePath = '/mnt/orderLog/ret'
    orderLog.to_pickle(os.path.join(savePath, 'orderLogWithRet_%s.pkl'%date))

/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (52,60) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are accounts with duplicated ticks:'

date      colo      accCode
20201106  zs_88_04  892402     11272
          zs_96_02  9655        1450
          zt_88_02  897102     20282
Name: ars, dtype: int64

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
81968,20201106,zs_94_03,9461,2002179.0,-1.0,7.0,50293.0,0.0,0.0,-1.0,-1.000000e+00
138507,20201106,zs_96_06,9758,2300653.0,-1.0,7.0,36673.0,0.0,0.0,-1.0,7.213082e+17
212878,20201106,zs_54_01,5470,2000565.0,-1.0,7.0,40926.0,0.0,0.0,-1.0,-1.000000e+00
214294,20201106,zs_54_01,5470,2000731.0,-1.0,1.0,53406.0,0.0,0.0,-1.0,8.689083e+08
216299,20201106,zs_54_01,5474,2300795.0,-1.0,1.0,36037.0,0.0,0.0,-1.0,8.689021e+08
221139,20201106,zs_54_01,5474,2300108.0,-1.0,1.0,47148.0,0.0,0.0,-1.0,8.689057e+08
226303,20201106,zs_94_05,9471,2000049.0,-1.0,7.0,35262.0,0.0,0.0,-1.0,-1.000000e+00
231049,20201106,zs_94_05,9471,2300724.0,-1.0,7.0,37303.0,0.0,0.0,-1.0,-1.000000e+00
484876,20201106,zs_64_01,6480,2002068.0,-1.0,1.0,35219.0,0.0,0.0,-1.0,2.825760e+05
493165,20201106,zs_64_01,6480,2002660.0,-1.0,1.0,51063.0,0.0,0.0,-1.0,4.023330e+05


...

Series([], Name: ars, dtype: int64)

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
83937,20201216,zs_54_01,5474,2002094.0,-1.0,7.0,36274.0,0.0,0.0,-1.0,-1.000000e+00
160339,20201216,zs_52_06,5269,2300410.0,-1.0,1.0,35172.0,0.0,0.0,-1.0,2.519470e+05
163475,20201216,zs_52_06,5269,2300521.0,-1.0,1.0,38454.0,0.0,0.0,-1.0,6.602330e+05
251992,20201216,zs_66_01,6634,2300272.0,-1.0,1.0,52636.0,0.0,0.0,-1.0,1.812810e+10
259507,20201216,zs_64_01,6480,2002729.0,-1.0,1.0,47305.0,0.0,0.0,-1.0,8.880200e+04
259615,20201216,zs_64_01,6480,2300151.0,-1.0,1.0,47491.0,0.0,0.0,-1.0,8.986500e+04
260830,20201216,zs_64_01,6480,2000677.0,-1.0,1.0,49418.0,0.0,0.0,-1.0,1.001710e+05
260910,20201216,zs_64_01,6480,2002956.0,-1.0,1.0,49543.0,0.0,0.0,-1.0,1.007390e+05
261181,20201216,zs_64_01,6480,2300445.0,-1.0,1.0,50128.0,0.0,0.0,-1.0,1.034090e+05
261199,20201216,zs_64_01,6480,2300789.0,-1.0,1.0,50137.0,0.0,0.0,-1.0,1.033610e+05


There are orders with all things same except sdd
        ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
679151   3688220.0  0.000419                  0.0         100.0   
679310   3686264.0   0.00044                  0.0         100.0   
681571  14404537.0  0.000657                  0.0         100.0   
681724  18252604.0  0.001406                  0.0         100.0   

        absOrderSizeCumFilled  accCode  ars  caamd  cancellationPending  \
679151                    0.0   966301  0.0    0.0                  0.0   
679310                    0.0   966301  0.0    0.0                  0.0   
681571                    0.0   966301  0.0    0.0                  0.0   
681724                    0.0   966301  0.0    0.0                  0.0   

              cfe         clock  clockAtArrival  cumSharesBought  \
679151  568630.46  1.608083e+15    1.608083e+15              0.0   
679310  506777.82  1.608083e+15    1.608083e+15            100.0   
681571  198186.36  1.608097e+15    

/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/work516/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
